# Initial visualization of GRACE mascons
## Dataset
GSFC mascons: [data](https://earth.gsfc.nasa.gov/geo/data/grace-mascons), [documentation](https://earth.gsfc.nasa.gov/sites/default/files/2022-05/gsfc_mascons_hdf5_format_rl06v2.pdf)

## Environments and packages


## to-do's
- [x] copy over updated function
- [ ] switch longitude from 0 - 360 to -180 - 180
- [ ] mascon_df to gdf

In [3]:
grace_filename = 'gsfc.glb_.200204_202211_rl06v2.0_obp-ice6gd.h5'
grace_filepath = '/global/scratch/users/ann_scheliga/'
func_dir = '/global/home/users/ann_scheliga/CYGNSS-GRACE/codebase/'

In [4]:
import numpy as np
import pandas as pd
import h5py
import sys
sys.path.append(func_dir)
import load_data
grace_dict = load_data.load_GRACE()

In [5]:

import numpy as np
import pandas as pd
import h5py
f = h5py.File(grace_filepath + grace_filename,'r')
grace_dict = dict()

land_subset = True
uncertainty = False
formatting = True

In [37]:
# MASCONS #
mascon_cols = list(f['mascon'])       # grab dataset names in mascon group
mascon_cols.remove('location_legend') # remove unused dataset name
mascon_df = pd.DataFrame()            # create empty pd.DataFrame
for key in mascon_cols:               # fill df
    mascon_df[key] = np.array(f['mascon'][key]).T.squeeze()
# Convert longitude from [0 to 360] to (-180 to 180]
mascon_df.loc[mascon_df['lon_center'] >180,'lon_center'] = mascon_df.loc[mascon_df['lon_center'] >180,'lon_center'] - 360
if land_subset:
    land_bool = mascon_df['location'] == 80
    mascon_df = mascon_df.loc[land_bool,:]
if formatting:
    mascon_df.index = mascon_df['labels'].astype(int)
grace_dict['mascon'] = mascon_df

In [7]:
# DATES #
# REFERENCE DATE (aka 2002 Jan 0)
start_date = pd.Timestamp('2001-12-31')
time_cols = list(f['time'])                # grab dataset names in time group
time_cols.remove('list_ref_days_solution') # remove unused dataset name
time_df = pd.DataFrame()                   # create empty pd.DataFrame for reference dates
for key in time_cols[2:-1]:                # fill df with days since reference day
    time_df[key] = np.array(f['time'][key]).T.squeeze()
date_df = time_df.apply(lambda x: pd.to_datetime(x, unit='D',origin=start_date),axis=1)
date_df.columns = ['date_first','date_last','date_middle']
date_df[['year_middle','doy_middle','frac_year_middle']] = pd.DataFrame(f['time']['yyyy_doy_yrplot_middle']).T
grace_dict['date'] = date_df

In [38]:
# CMWE SOLUTIONS #
cmwe = pd.DataFrame(f['solution']['cmwe'])
if land_subset:
    cmwe = cmwe.loc[land_bool,:]
if formatting:
    cmwe.columns = date_df['date_middle']
    cmwe.index = mascon_df['labels'].astype(int)
grace_dict['cmwe'] = cmwe

In [9]:
# UNCERTAINTY #
if uncertainty:
    uncertainty_cols = list(f['uncertainty'])
    uncertainty_df = pd.DataFrame()
    for key in uncertainty_cols[:-1]:
        uncertainty_df[key] = np.array(f['uncertainty'][key]).T.squeeze()
    noise_df = pd.DataFrame(f['uncertainty']['noise_2sigma'])
    uncertainty_df = pd.concat([uncertainty_df, noise_df],axis=1)
    if land_subset:
        uncertainty_df = uncertainty_df.loc[land_bool,:]
    if formatting:
        mascon_df.index = mascon_df['labels'].astype(int)
    grace_dict['uncertainty'] = uncertainty_df